# Lecture 2 Hands-On: Queueing Models

This notebook accompanies Lecture 2. We revisit Markovian queues, implement simulators for M/M/1 and M/M/c, and explore a non-Markov queue (M/G/1 with lognormal service).

## Imports

In [ ]:

import math
import itertools

import numpy as np
import pandas as pd
import simpy
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-darkgrid')
RNG = np.random.default_rng(2025)


---
## Part A — Birth–Death Refresher

Use this section to verify the analytical expressions for the stationary measure of a homogeneous birth-death process discussed in class.

In [ ]:

def geometric_stationary(lam, mu , n):
    return None

# Example: lam = 4 per hour, mu = 6 per hour
pi = np.array([geometric_stationary(lam, mu, n) for n in range(10)])
pi.sum()


### Task A1
Compute $L, L_q, W, W_q$ for $(\lambda, \mu) = (4, 6)$ using the formulas from lecture. Then cross-check with Little's Law ($L = \lambda W$, $L_q = \lambda W_q$).

In [ ]:

lam, mu = 4.0, 6.0
rho = lam / mu
L_formula = # TODO
Lq_formula = # TODO
W_formula = # TODO
Wq_formula = # TODO

{
    'rho': rho,
    'L': L_formula,
    'L_q': Lq_formula,
    'W': W_formula,
    'W_q': Wq_formula,
    'Little_L': # TODO,
    'Little_Lq': # TODO,
}


---
## Part B — Plain-Python Simulation of M/M/c

We extend the Part 1 simulator to $c$ servers. Below is a plain-Python discrete-event simulator that assigns each arrival to the server that becomes free the earliest (FCFS with $c$ identical servers).

In [ ]:

def simulate_mmc_basic(lambda_rate, mu_rate, servers, sim_hours, rng, max_events=10_000):
    arrival_times = []
    service_start_times = []
    departure_times = []
    waiting_times = []
    system_times = []

    current_time = 0.0
    server_available_times = [0.0] * servers
    event_count = 0

    while current_time < sim_hours and event_count < max_events:
        inter_arrival = rng.exponential(1 / lambda_rate)
        current_time += inter_arrival
        if current_time > sim_hours:
            break

        arrival_times.append(current_time)

        # Choose the server that becomes free the earliest (FCFS across servers)
        server_idx = int(np.argmin(server_available_times))
        earliest_completion = server_available_times[server_idx]

        service_time = rng.exponential(1 / mu_rate)
        start_time = max(current_time, earliest_completion)
        finish_time = start_time + service_time

        server_available_times[server_idx] = finish_time

        service_start_times.append(start_time)
        departure_times.append(finish_time)
        waiting_times.append(start_time - current_time)
        system_times.append(finish_time - current_time)

        event_count += 1

    return {
        'arrivals': np.array(arrival_times),
        'starts': np.array(service_start_times),
        'departures': np.array(departure_times),
        'waiting_times': np.array(waiting_times),
        'system_times': np.array(system_times),
    }

# Choose parameters (run long enough to reach steady behaviour)
lam_mm2, mu_mm2, servers = 8.0, 5.0, 2
logs_mmc = simulate_mmc_basic(lam_mm2, mu_mm2, servers, sim_hours=2000.0, rng=RNG)
print(f"Simulated {len(logs_mmc['arrivals'])} arrivals.")

### Task B1
Validate the simulation by comparing empirical means with the Erlang-C predictions ($P_{wait}$, $L_q$, $W_q$). Provide a short discussion on discrepancies and simulation length.

In [ ]:

def erlang_c(lambda_rate, mu_rate, servers):
    rho = lambda_rate / (servers * mu_rate)
    if rho >= 1:
        raise ValueError('System is unstable (rho >= 1).')
    terms = [((servers * rho) ** k) / math.factorial(k) for k in range(servers)]
    numerator = ((servers * rho) ** servers) / math.factorial(servers) * (1 / (1 - rho))
    denominator = sum(terms) + numerator
    P_wait = numerator / denominator
    L_q = P_wait * (rho / (1 - rho))
    W_q = L_q / lambda_rate
    W = W_q + 1 / mu_rate
    L = L_q + (lambda_rate / mu_rate)
    return {'rho': rho, 'P_wait': P_wait, 'L_q': L_q, 'L': L, 'W_q': W_q, 'W': W}


In [ ]:

theory_mmc = erlang_c(lam_mm2, mu_mm2, servers)
waiting_emp = logs_mmc['waiting_times']
system_emp = logs_mmc['system_times']

comparison = {
    'rho': theory_mmc['rho'],
    'P_wait_theory': theory_mmc['P_wait'],
    'P_wait_emp': float((waiting_emp > 1e-12).mean()),
    'W_q_theory': theory_mmc['W_q'],
    'W_q_emp': float(waiting_emp.mean()),
    'W_theory': theory_mmc['W'],
    'W_emp': float(system_emp.mean()),
}
comparison


---
## Part C — SimPy M/M/c

Build on the primer from Lecture 1: we now use a `simpy.Resource(capacity=c)` to model $c$ servers and reproduce the M/M/$c$ queue.

### SimPy M/M/c Implementation
We now implement an M/M/c queue using SimPy's `Resource(capacity=c)`, aligned with the primer from Lecture 1. We log arrivals, service starts, and departures to compute waiting/sojourn times and compare to Erlang-C.

In [ ]:

def simulate_mmc_simpy(lambda_rate, mu_rate, servers, duration_hours, rng, max_customers=100_000):
    env = simpy.Environment()
    server = simpy.Resource(env, capacity=servers)

    arrivals, starts, departures = [], [], []

    def customer(env, cust_id):
        # TODO

    def arrival_generator(env):
        # TODO

    # TODO start the simulation

    a = np.array(arrivals)
    s = np.array(starts)
    d = np.array(departures)
    # TODO  align lengths of a, s, d

    return {
        'arrivals': a,
        'starts': s,
        'departures': d,
        'waiting_times': s - a,
        'system_times': d - a,
    }

# Example run and comparison
lam_c, mu_c, c_servers = 8.0, 5.0, 2
sim_hours = 200.0
logs_c = simulate_mmc_simpy(lam_c, mu_c, c_servers, sim_hours, RNG)
theory_c = erlang_c(lam_c, mu_c, c_servers)

wait = logs_c['waiting_times']
sys_t = logs_c['system_times']
a, d = logs_c['arrivals'], logs_c['departures']
m = a.size
# TODO Discard initial and final 10% and 5% of data to reduce initialization and ending effects
front = # TODO
back = # TODO
if front or back:
    sl = slice(front, m - back if back else None)
    a, d = a[sl], d[sl]
    wait, sys_t = wait[sl], sys_t[sl]
lam_hat = (d.size - 1) / (d[-1] - d[0]) if d.size > 1 else np.nan

rho_emp = lam_hat / (c_servers * mu_c) if np.isfinite(lam_hat) else np.nan
L_q_emp = float(lam_hat * wait.mean()) if np.isfinite(lam_hat) else np.nan
L_emp = float(lam_hat * sys_t.mean()) if np.isfinite(lam_hat) else np.nan

results_c = {
    'rho_theory': theory_c['rho'],
    'rho_emp': rho_emp,
    'P_wait_theory': theory_c['P_wait'],
    'P_wait_emp': float((wait > 1e-12).mean()),
    'W_q_theory': theory_c['W_q'],
    'W_q_emp': float(wait.mean()),
    'W_theory': theory_c['W'],
    'W_emp': float(sys_t.mean()),
    'L_q_theory': theory_c['L_q'],
    'L_q_emp': L_q_emp,
    'L_theory': theory_c['L'],
    'L_emp': L_emp,
}
results_c


---
## Part D — Non-Markov Queue (M/G/1 with Lognormal Service)

We consider an M/G/1 queue where service times are lognormally distributed (mean $1/\mu$, variance controlled by $\sigma$). This breaks the memoryless assumption and highlights the impact of variability.

In [ ]:

def simulate_mg1_lognormal(lambda_rate, service_mean, service_sigma, duration_hours, rng):
    env = simpy.Environment()
    server = simpy.Resource(env, capacity=1)

    arrivals, starts, departures = [], [], []

    def service_time():
        mu_log = math.log(service_mean) - 0.5 * service_sigma ** 2
        return math.exp(mu_log + service_sigma * rng.normal())

    def customer(env, cust_id):
        arrival = env.now
        arrivals.append(arrival)
        with server.request() as req:
            yield req
            start = env.now
            starts.append(start)
            yield env.timeout(service_time())
            departures.append(env.now)

    def generator(env):
        cust_id = 0
        while True:
            inter = rng.exponential(1 / lambda_rate)
            yield env.timeout(inter)
            cust_id += 1
            if env.now > duration_hours:
                break
            env.process(customer(env, cust_id))

    env.process(generator(env))
    env.run(until=duration_hours)

    return {
        'arrivals': np.array(arrivals),
        'starts': np.array(starts),
        'departures': np.array(departures),
    }


### Task D1
Run the simulator for several hours with $\lambda = 4$, service mean $1/5$, and lognormal shape parameter $\sigma = 0.6$. Compute empirical waiting times and compare the mean to the Pollaczek–Khinchine prediction.

In [ ]:

lambda_mg1 = 4.0
service_mean = 1 / 5.0
service_sigma = 0.6
duration = 8.0
logs_mg1 = simulate_mg1_lognormal(lambda_mg1, service_mean, service_sigma, duration, RNG)

# TODO Fix the different lengths
waiting_mg1 = logs_mg1['starts'] - logs_mg1['arrivals']
system_mg1 = logs_mg1['departures'] - logs_mg1['arrivals']

# TODO Discard initial and final 10% and 5% of data to reduce initialization and ending effects

mean_wait_emp = waiting_mg1.mean()
var_service = (math.exp(service_sigma ** 2) - 1) * (service_mean ** 2)
second_moment_service = var_service + service_mean ** 2
rho_mg1 = lambda_mg1 * service_mean
Wq_pk = # TODO
W_pk = # TODO

{
    'rho': rho_mg1,
    'empirical_wait_mean': mean_wait_emp,
    'PK_wait_mean': Wq_pk,
    'empirical_sojourn_mean': system_mg1.mean(),
    'PK_sojourn_mean': W_pk,
}


### Task D2
Produce plots illustrating the difference between exponential and lognormal services:
- Histogram (or KDE) of waiting times for exponential vs. lognormal service under the same arrival rate.
- Running time-average of waiting times to assess convergence.
- Empirical tail probability $\mathbb{P}(W_q > t)$ vs. $t$, compared with exponential-case prediction.

### Task D3
Quantitative bounds: for $N$ samples of waiting time, derive a confidence interval for the mean using either CLT or Bernstein-style inequality. Compare the width of the interval for exponential vs. lognormal services.

---
## Part E — To Explore Further

- Extend the M/G/1 example to a GI/G/1 queue with Erlang inter-arrivals; observe how burstiness affects $W_q$.
- Implement regenerative confidence intervals (batch means) to reduce bias.
- Try a heavy-tailed service time (Pareto) and examine the empirical tail behaviour.